In [1]:
from huggingface_hub import login

# Login using your Hugging Face account token
login(token="YOUR_HF_TOKEN")
print("Logged in successfully!")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Logged in successfully!


In [2]:
!pip install torch


[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# ✅ Enable CUDA for faster loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load LLaMA tokenizer and model
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

llama_model.to(device)

# Set padding token
if llama_tokenizer.pad_token is None:
    llama_tokenizer.pad_token = llama_tokenizer.eos_token


/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1738877927.220128      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


In [47]:
def generate_response(user_question, sentiment,frnd,expl):
    #input_text = f"User name: {frnd} User description: {expl}User prompt: {user_question} . feeling detected {sentiment}. Respond in a supportive, brief, and conversational tone. Offer helpful advice or jokes where appropriate, without a formal therapist-like approach. Focus on uplifting the user with empathy and positive suggestions, keeping the response clear and concise. Do not assume user input beyond the indicated sentiment. The response should feel like a friendly chatbot conversation. donot give additional notes or alternative responses. word limit 300 words. give your response after RESPONSE: keyword only"
    input_text= f"""You are a friendly AI counselor. Respond in a warm, supportive, and conversational tone. Do not assume beyond the given input or create user prompts. No extra notes or alternatives. Keep it ≤300 words. Reply like a chatbot, not a therapist. Respond only after "RESPONSE:" keyword if necessary.  
analyse the prompt and give suggestions how to overcome that problem. keep your response in a plain text paragraph. 
User: {frnd} | Info: {expl} | Prompt: {prpt} | Emotion: {emotion} ({confidence}) .. """ 

    input_ids = llama_tokenizer(input_text, return_tensors='pt').input_ids.to(device)

    output =llama_model.generate(
        input_ids, 
        max_length=400,  # ✅ Increase max tokens
        num_return_sequences=1,  
        temperature=0.7,  # ✅ More creative output
        top_k=50,  # ✅ Sample diverse words
        top_p=0.9,  
        do_sample=True,  # ✅ Prevents deterministic outputs
        pad_token_id=llama_tokenizer.eos_token_id  # ✅ Avoids abrupt cutoff
    )

    return llama_tokenizer.decode(output[0], skip_special_tokens=True)



In [ ]:
from transformers import pipeline
# Load the emotion classification pipeline
emotion_model = pipeline("text-classification", model="bhadresh-savani/bert-base-go-emotion", return_all_scores=True, device=device)
def get_prominent_emotion(text):
    emotions = emotion_model(text)[0]  # Get scores for all emotions
    top_emotion = max(emotions, key=lambda x: x['score'])  # Find the highest score
    return top_emotion['label'], top_emotion['score']


Device set to use cpu


In [48]:


def generate(user_question,frnd,expl):
    # Get the top emotion
    emotion, confidence = get_prominent_emotion(user_question)
    print(f"Emotion: {emotion} (Confidence: {confidence:.2f})")
    response = generate_response(user_question, f"Emotion: {emotion} (Confidence: {confidence:.2f})",frnd,expl)
    try:
        response_text = response.split("RESPONSE:")[2].strip()
        l = response_text.split("\n")
        l = '\n'.join(l[2:])
        print(f"Dost: {response_text}")
    except:
        
        response_text = response.split("..").strip()
        print(f"Dost: {response_text}")


In [40]:
print("Welcome to your own A.I. counsellor. type:leave to exit the program")
frnd = input("What is your name? :")
expl = input("Explain yourself breifly")
while(True):
    prpt = input("You: ")
    if prpt == "leave":break;
    generate(prpt,frnd,expl)
    
    
    

Welcome to your own A.I. counsellor. type:leave to exit the program


What is your name? : Maneet
Explain yourself breifly College Student
You:  I feel very Bad today ;-;


Emotion: sadness (Confidence: 0.34)
Dost: I can tell you're having a tough day, Maneet. It sounds like sadness is really weighing on you. Would you like to talk about what's on your mind and why you're feeling this way? Sometimes sharing what's bothering you can help you feel a bit better. Is there something specific that's going on or has been going on that's causing you distress?


You:  my friends bully me too much these days. i feel insecure


Emotion: sadness (Confidence: 0.27)
Dost: Sorry to hear that Maneet. It can be really tough to deal with bullying, especially when it's from people you consider friends. Feeling insecure can be a really challenging emotion to navigate. Here are some suggestions that might help: Have you talked to your friends about how their behavior is making you feel? Sometimes, people don't realize the impact of their words or actions, and talking it out can help clear the air. You might also consider talking to a trusted adult, like a teacher, counselor, or family member, about what's going on. They can offer you support and help you figure out ways to address the situation. Additionally, it might be helpful to surround yourself with people who support and uplift you. Joining a club or group that aligns with your interests can be a great way to meet new people who share your


You:  leave


In [ ]:
print("Welcome to your own A.I. counsellor. type:leave to exit the program")
frnd = input("What is your name? :")
expl = input("Explain yourself breifly")
print()
while(True):
    prpt = input("You: ")
    if prpt == "leave":break;
    print()
    generate(prpt,frnd,expl)

Welcome to your own A.I. counsellor. type:leave to exit the program


What is your name? : Rohit
Explain yourself breifly A war Veteran


You:  I lost my limb in pakistani war. i still have nightmares of that day.



Emotion: sadness (Confidence: 0.46)
Dost: " keyword, which was not present in the original message. I'll make sure to follow the correct protocol in the future. I'll assume that I should only respond when the "
